# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
# %pip install duckdb
# %pip install jupysql
# %pip install duckdb-engine

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [3]:
dataset_file = 'datos/competencia_01_brandoni.csv'

In [4]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Definir la macro para sumar sin valores nulos


In [5]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

,Success


Crear nueva tabla con todos los atributos originales más los nuevos de ingeniería de variables

https://chatgpt.com/share/66edc29b-e0a4-8004-aa86-508d1e7eefa5


In [6]:
%%sql
CREATE OR REPLACE TABLE competencia_01_fe_agr_1 AS
WITH base_features AS (
    SELECT
        *,
        suma_sin_null(Master_msaldototal, Visa_msaldototal) AS tc_saldo_total,
        suma_sin_null(mcuenta_corriente, mcaja_ahorro) AS saldo_total_cuentas,
        suma_sin_null(
            suma_sin_null(Visa_msaldototal, Master_msaldototal),
            suma_sin_null(mcuenta_corriente, mcaja_ahorro)
        ) AS saldo_total_completo,
        suma_sin_null(
            mprestamos_personales,
            suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios)
        ) AS deuda_total_prestamos,

        LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpayroll,
        LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpayroll,
        LAG(mpayroll, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpayroll,

        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_tarjetas_consumo,

        LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cuenta_corriente,
        LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cuenta_corriente,
        LAG(mcuenta_corriente, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_cuenta_corriente,

        LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_caja_ahorro,
        LAG(mcaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_caja_ahorro,
        LAG(mcaja_ahorro, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_caja_ahorro,

        LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cuentas_saldo,
        LAG(mcuentas_saldo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cuentas_saldo,
        LAG(mcuentas_saldo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_cuentas_saldo,

        LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_comisiones_mantenimiento,

        LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_comisiones_otras,
        LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_comisiones_otras,
        LAG(mcomisiones_otras, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_comisiones_otras,

        mpayroll - LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mpayroll,
        mpayroll - LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mpayroll,
        mpayroll - LAG(mpayroll, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_mpayroll,

        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_visa_consumo,

        mcuenta_corriente - LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_cuenta_corriente,

        mprestamos_personales - LAG(mprestamos_personales, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_prestamos_personales,
        mprestamos_personales - LAG(mprestamos_personales, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_prestamos_personales,
        mprestamos_personales - LAG(mprestamos_personales, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_prestamos_personales,

        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_transacciones_visa,

        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_comisiones_mantenimiento,

        mcomisiones_otras - LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_comisiones_otras,

        REGR_SLOPE(mpayroll, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_1_cliente_antiguedad,

        REGR_SLOPE(mpayroll, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_2_cliente_antiguedad,

        REGR_SLOPE(mpayroll, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_3_cliente_antiguedad,

        REGR_SLOPE(mcuenta_corriente, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_1_cliente_antiguedad,

        REGR_SLOPE(mcuenta_corriente, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_2_cliente_antiguedad,

        REGR_SLOPE(mcuenta_corriente, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_3_cliente_antiguedad,

        REGR_SLOPE(mtarjeta_visa_consumo, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_1_cliente_antiguedad,

        REGR_SLOPE(mtarjeta_visa_consumo, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_2_cliente_antiguedad,

        REGR_SLOPE(mtarjeta_visa_consumo, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_3_cliente_antiguedad,

        REGR_SLOPE(mcomisiones_mantenimiento, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_1_cliente_antiguedad,

        REGR_SLOPE(mcomisiones_mantenimiento, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_2_cliente_antiguedad,

        REGR_SLOPE(mcomisiones_mantenimiento, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_3_cliente_antiguedad,

        REGR_SLOPE(mcomisiones_otras, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_1_cliente_antiguedad,

        REGR_SLOPE(mcomisiones_otras, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_2_cliente_antiguedad,

        REGR_SLOPE(mcomisiones_otras, cliente_antiguedad) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_3_cliente_antiguedad,

       REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_1_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_2_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_3_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_1_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_2_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_3_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_1_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_2_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_3_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_1_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_2_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_3_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_1_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_2_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_3_foto_mes,

        CASE WHEN Master_status IN (6, 7, 9) THEN 1 ELSE 0 END AS Master_cuenta_cerrada,
        CASE WHEN Visa_status IN (6, 7, 9) THEN 1 ELSE 0 END AS Visa_cuenta_cerrada,

        mcuentas_saldo / NULLIF(suma_sin_null(mactivos_margen, mpasivos_margen), 0) AS ratio_saldo_margen,
        suma_sin_null(mcomisiones_mantenimiento, mcomisiones_otras) AS total_comisiones_cliente,

        suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) AS transacciones_totales_tarjetas,
        ctarjeta_debito_transacciones - suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) AS delta_debito_vs_credito,
        thomebanking + tmobile_app AS indicador_banca_digital

    FROM competencia_01
)

SELECT
    *,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cliente_antiguedad) AS cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mpayroll) AS payroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_visa_consumo) AS visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ctarjeta_visa) AS visa_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_master_consumo) AS master_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ctarjeta_master) AS master_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cproductos) AS productos_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mcuenta_corriente) AS cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mcaja_ahorro) AS caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY suma_sin_null(ctarjeta_visa, ctarjeta_master)) AS tarjetas_credito_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo)) AS tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mprestamos_personales) AS prestamos_personales_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mprestamos_hipotecarios) AS prestamos_hipotecarios_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY tc_saldo_total) AS tc_saldo_total_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY saldo_total_cuentas) AS saldo_total_cuentas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY saldo_total_completo) AS saldo_total_completo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY deuda_total_prestamos) AS deuda_total_prestamos_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_mpayroll) AS lag_1_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_mpayroll) AS lag_2_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_mpayroll) AS lag_3_mpayroll_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_tarjetas_consumo) AS lag_1_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_tarjetas_consumo) AS lag_2_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_tarjetas_consumo) AS lag_3_tarjetas_consumo_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_cuenta_corriente) AS lag_1_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_cuenta_corriente) AS lag_2_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_cuenta_corriente) AS lag_3_cuenta_corriente_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_caja_ahorro) AS lag_1_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_caja_ahorro) AS lag_2_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_caja_ahorro) AS lag_3_caja_ahorro_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_cuentas_saldo) AS lag_1_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_cuentas_saldo) AS lag_2_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_cuentas_saldo) AS lag_3_cuentas_saldo_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_comisiones_mantenimiento) AS lag_1_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_comisiones_mantenimiento) AS lag_2_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_comisiones_mantenimiento) AS lag_3_comisiones_mantenimiento_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_comisiones_otras) AS lag_1_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_comisiones_otras) AS lag_2_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_comisiones_otras) AS lag_3_comisiones_otras_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_mpayroll) AS delta_1_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_mpayroll) AS delta_2_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_mpayroll) AS delta_3_mpayroll_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_visa_consumo) AS delta_1_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_visa_consumo) AS delta_2_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_visa_consumo) AS delta_3_visa_consumo_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_cuenta_corriente) AS delta_1_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_cuenta_corriente) AS delta_2_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_cuenta_corriente) AS delta_3_cuenta_corriente_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_prestamos_personales) AS delta_1_prestamos_personales_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_prestamos_personales) AS delta_2_prestamos_personales_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_prestamos_personales) AS delta_3_prestamos_personales_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_transacciones_visa) AS delta_1_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_transacciones_visa) AS delta_2_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_transacciones_visa) AS delta_3_transacciones_visa_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_comisiones_mantenimiento) AS delta_1_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_comisiones_mantenimiento) AS delta_2_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_comisiones_mantenimiento) AS delta_3_comisiones_mantenimiento_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_comisiones_otras) AS delta_1_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_comisiones_otras) AS delta_2_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_comisiones_otras) AS delta_3_comisiones_otras_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_1_cliente_antiguedad) AS payroll_slope_1_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_2_cliente_antiguedad) AS payroll_slope_2_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_3_cliente_antiguedad) AS payroll_slope_3_cliente_antiguedad_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_1_cliente_antiguedad) AS cuenta_corriente_slope_1_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_2_cliente_antiguedad) AS cuenta_corriente_slope_2_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_3_cliente_antiguedad) AS cuenta_corriente_slope_3_cliente_antiguedad_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_1_cliente_antiguedad) AS visa_consumo_slope_1_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_2_cliente_antiguedad) AS visa_consumo_slope_2_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_3_cliente_antiguedad) AS visa_consumo_slope_3_cliente_antiguedad_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_1_cliente_antiguedad) AS comisiones_mantenimiento_slope_1_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_2_cliente_antiguedad) AS comisiones_mantenimiento_slope_2_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_3_cliente_antiguedad) AS comisiones_mantenimiento_slope_3_cliente_antiguedad_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_1_cliente_antiguedad) AS comisiones_otras_slope_1_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_2_cliente_antiguedad) AS comisiones_otras_slope_2_cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_3_cliente_antiguedad) AS comisiones_otras_slope_3_cliente_antiguedad_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_1_foto_mes) AS payroll_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_2_foto_mes) AS payroll_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_3_foto_mes) AS payroll_slope_3_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_1_foto_mes) AS cuenta_corriente_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_2_foto_mes) AS cuenta_corriente_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_3_foto_mes) AS cuenta_corriente_slope_3_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_1_foto_mes) AS visa_consumo_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_2_foto_mes) AS visa_consumo_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_3_foto_mes) AS visa_consumo_slope_3_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_1_foto_mes) AS comisiones_mantenimiento_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_2_foto_mes) AS comisiones_mantenimiento_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_3_foto_mes) AS comisiones_mantenimiento_slope_3_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_1_foto_mes) AS comisiones_otras_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_2_foto_mes) AS comisiones_otras_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_3_foto_mes) AS comisiones_otras_slope_3_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ratio_saldo_margen) AS ratio_saldo_margen_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY total_comisiones_cliente) AS total_comisiones_cliente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY transacciones_totales_tarjetas) AS transacciones_totales_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_debito_vs_credito) AS delta_debito_vs_credito_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY indicador_banca_digital) AS indicador_banca_digital_10tile

FROM base_features;


RuntimeError: (duckdb.IOException) IO Error: Could not write file ".tmp\duckdb_temp_block-4611686018428745078.block" (error in WriteFile): Espacio en disco insuficiente.

[SQL: CREATE OR REPLACE TABLE competencia_01_fe_agr_1 AS
WITH base_features AS (
    SELECT
        *,
        suma_sin_null(Master_msaldototal, Visa_msaldototal) AS tc_saldo_total,
        suma_sin_null(mcuenta_corriente, mcaja_ahorro) AS saldo_total_cuentas,
        suma_sin_null(
            suma_sin_null(Visa_msaldototal, Master_msaldototal),
            suma_sin_null(mcuenta_corriente, mcaja_ahorro)
        ) AS saldo_total_completo,
        suma_sin_null(
            mprestamos_personales,
            suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios)
        ) AS deuda_total_prestamos,

        LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpayroll,
        LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpayroll,
        LAG(mpayr

Y a no olvidarse guardar las nueva tabla

In [7]:
%%sql
COPY competencia_01_fe_agr_3 TO 'datos/competencia_01_brandoni_fe_agr_3.csv' (FORMAT CSV, HEADER TRUE);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
